In [2]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [3]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()
# importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:08<02:12,  1.37it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:42<00:37,  2.63it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [00:53<00:23,  2.91it/s]

...

 79%|███████▉  | 158/200 [01:06<00:14,  2.93it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:25<00:00,  2.33it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 11147.89it/s]


In [7]:
val_data = train[train['date_id']>385].copy().reset_index()
val_data

,index,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,stats
0,4192980,0,386,0,2555434.64,-1,1.000032,9966697.63,NaN,NaN,0.999746,18840.60,1.000032,2408.10,1.000000,-2.340078,21230,386_0_0,"[0.0, 2555434.64, -1.0, 1.000032, 9966697.63, ..."
1,4192981,1,386,0,274697.82,1,0.999714,1324398.60,NaN,NaN,0.999370,71153.73,1.000002,174.08,1.000000,-8.130074,21230,386_0_1,"[0.0, 274697.82, 1.0, 0.999714, 1324398.6, -1...."
2,4192982,2,386,0,415532.77,1,0.999826,2463160.51,NaN,NaN,0.999323,46315.74,1.000328,22484.74,1.000000,0.790358,21230,386_0_2,"[0.0, 415532.77, 1.0, 0.999826, 2463160.51, -1..."
3,4192983,3,386,0,13491906.43,-1,1.000171,27369686.77,NaN,NaN,0.999774,27757.32,1.000121,14889.54,1.000000,6.059408,21230,386_0_3,"[0.0, 13491906.43, -1.0, 1.000171, 27369686.77..."
4,4192984,4,386,0,2097475.00,1,0.999980,15853553.29,NaN,NaN,0.999869,16029.79,1.000147,18016.00,1.000000,0.499487,21230,386_0_4,"[0.0, 2097475.0, 1.0, 0.99998, 15853553.29, -1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044885,5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195,"[540.0, 2440722.89, -1.0, 1.000317, 28280361.7..."
1044886,5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196,"[540.0, 349510.47, -1.0, 1.000643, 9187699.11,..."
1044887,5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197,"[540.0, 0.0, 0.0, 0.995789, 12725436.1, 0.9957..."
1044888,5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198,"[540.0, 1000898.84, 1.0, 0.99921, 94773271.05,..."


In [8]:
stats_cols =  ['seconds_in_bucket','imbalance_size','imbalance_buy_sell_flag','reference_price','matched_size','far_price','near_price','bid_price','bid_size','ask_price','ask_size','wap']

In [9]:
x =torch.rand(10)
y= torch.rand(9)

In [10]:
# torch.stack([x,y])

In [11]:
val_data['stats']  = pd.Series(val_data[stats_cols].fillna(-1).values.tolist())

In [12]:
val_data['stats']

0          [0.0, 2555434.64, -1.0, 1.000032, 9966697.63, ...
1          [0.0, 274697.82, 1.0, 0.999714, 1324398.6, -1....
2          [0.0, 415532.77, 1.0, 0.999826, 2463160.51, -1...
3          [0.0, 13491906.43, -1.0, 1.000171, 27369686.77...
4          [0.0, 2097475.0, 1.0, 0.99998, 15853553.29, -1...
                                 ...                        
1044885    [540.0, 2440722.89, -1.0, 1.000317, 28280361.7...
1044886    [540.0, 349510.47, -1.0, 1.000643, 9187699.11,...
1044887    [540.0, 0.0, 0.0, 0.995789, 12725436.1, 0.9957...
1044888    [540.0, 1000898.84, 1.0, 0.99921, 94773271.05,...
1044889    [540.0, 1884285.71, -1.0, 1.002129, 24073677.3...
Name: stats, Length: 1044890, dtype: object

In [13]:
val_data[stats_cols]

,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap
0,0,2555434.64,-1,1.000032,9966697.63,NaN,NaN,0.999746,18840.60,1.000032,2408.10,1.000000
1,0,274697.82,1,0.999714,1324398.60,NaN,NaN,0.999370,71153.73,1.000002,174.08,1.000000
2,0,415532.77,1,0.999826,2463160.51,NaN,NaN,0.999323,46315.74,1.000328,22484.74,1.000000
3,0,13491906.43,-1,1.000171,27369686.77,NaN,NaN,0.999774,27757.32,1.000121,14889.54,1.000000
4,0,2097475.00,1,0.999980,15853553.29,NaN,NaN,0.999869,16029.79,1.000147,18016.00,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1044885,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328
1044886,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819
1044887,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797
1044888,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008


In [14]:
val_data[stats_cols].fillna(-1)

,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap
0,0,2555434.64,-1,1.000032,9966697.63,-1.000000,-1.000000,0.999746,18840.60,1.000032,2408.10,1.000000
1,0,274697.82,1,0.999714,1324398.60,-1.000000,-1.000000,0.999370,71153.73,1.000002,174.08,1.000000
2,0,415532.77,1,0.999826,2463160.51,-1.000000,-1.000000,0.999323,46315.74,1.000328,22484.74,1.000000
3,0,13491906.43,-1,1.000171,27369686.77,-1.000000,-1.000000,0.999774,27757.32,1.000121,14889.54,1.000000
4,0,2097475.00,1,0.999980,15853553.29,-1.000000,-1.000000,0.999869,16029.79,1.000147,18016.00,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1044885,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328
1044886,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819
1044887,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797
1044888,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008


In [ ]:
val_data[stats_cols].fillna(-1).values

In [ ]:
val_data[stats_cols].fillna(-1).values.tolist()

In [17]:
pd.Series(val_data[stats_cols].fillna(-1).values.tolist())

0          [0.0, 2555434.64, -1.0, 1.000032, 9966697.63, ...
1          [0.0, 274697.82, 1.0, 0.999714, 1324398.6, -1....
2          [0.0, 415532.77, 1.0, 0.999826, 2463160.51, -1...
3          [0.0, 13491906.43, -1.0, 1.000171, 27369686.77...
4          [0.0, 2097475.0, 1.0, 0.99998, 15853553.29, -1...
                                 ...                        
1044885    [540.0, 2440722.89, -1.0, 1.000317, 28280361.7...
1044886    [540.0, 349510.47, -1.0, 1.000643, 9187699.11,...
1044887    [540.0, 0.0, 0.0, 0.995789, 12725436.1, 0.9957...
1044888    [540.0, 1000898.84, 1.0, 0.99921, 94773271.05,...
1044889    [540.0, 1884285.71, -1.0, 1.002129, 24073677.3...
Length: 1044890, dtype: object

In [18]:
val_data['stats']

0          [0.0, 2555434.64, -1.0, 1.000032, 9966697.63, ...
1          [0.0, 274697.82, 1.0, 0.999714, 1324398.6, -1....
2          [0.0, 415532.77, 1.0, 0.999826, 2463160.51, -1...
3          [0.0, 13491906.43, -1.0, 1.000171, 27369686.77...
4          [0.0, 2097475.0, 1.0, 0.99998, 15853553.29, -1...
                                 ...                        
1044885    [540.0, 2440722.89, -1.0, 1.000317, 28280361.7...
1044886    [540.0, 349510.47, -1.0, 1.000643, 9187699.11,...
1044887    [540.0, 0.0, 0.0, 0.995789, 12725436.1, 0.9957...
1044888    [540.0, 1000898.84, 1.0, 0.99921, 94773271.05,...
1044889    [540.0, 1884285.71, -1.0, 1.002129, 24073677.3...
Name: stats, Length: 1044890, dtype: object

In [19]:
# trading_data = torch_classes.TradingData()

In [20]:
trading_data.fill_hidden_states_for_test(model_data['db_train'])
trading_data.reset_hidden(64,device='cuda:0')

0
tensor([[ 0.0345, -0.2014, -0.0973, -0.1866,  0.0164,  0.0504, -0.0959, -0.1377,
         -0.1507,  0.1304, -0.3100, -0.0202, -0.3681,  0.0470, -0.0590, -0.1163,
         -0.2180,  0.5155, -0.1729, -0.4450,  0.0609, -0.0821, -0.2926, -0.1686,
         -0.3532, -0.1522,  0.1040, -0.0136,  0.2139,  0.3712,  0.2729, -0.1115,
         -0.1609, -0.2771,  0.0725,  0.2263, -0.0606,  0.3594, -0.3719,  0.0435,
          0.1348,  0.1076,  0.2491, -0.0524, -0.2780,  0.0128, -0.2476, -0.0966,
          0.1312, -0.0224, -0.3147,  0.1783, -0.3575, -0.3230, -0.0867, -0.1760,
          0.0042, -0.2829,  0.1796, -0.2797,  0.2116,  0.3598, -0.1037,  0.1804]],
       device='cuda:0')
1
tensor([[ 0.0231, -0.0883,  0.0912, -0.3205,  0.2108,  0.1581, -0.1691,  0.0587,
         -0.2095,  0.1421, -0.4246,  0.1013, -0.0568, -0.0108,  0.3418, -0.3031,
         -0.4044,  0.4295, -0.0914, -0.3270, -0.2582,  0.2438,  0.0045, -0.1414,
         -0.4322, -0.1899,  0.1431, -0.3225,  0.0979,  0.3287,  0.1995, -0.1184

In [21]:
criterion = nn.L1Loss()

In [22]:
model = model.to('cuda:0')

In [23]:
import importlib

In [50]:
importlib.reload(training_testing)
importlib.reload(torch_classes)

<module 'torch_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\torch_classes.py'>

In [51]:
model = torch_classes.GRUNetV2(12,64)
model_loc = f"C:/Users/Nick/Documents/GitHub/OptiverKaggle/models/fresh-puddle-114/fresh-puddle-114_200.pt"
model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
print(model_data['model_state_dict'].keys())
model.load_state_dict(model_data['model_state_dict'], strict=False)
model = model.to(device='cuda:0')

odict_keys(['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [25]:
stock_ids = trading_data.val_stock_batches[0]

In [61]:
output,relu = training_testing.validate_model(trading_data,model,criterion,100)

tensor([ 0.0231, -0.0367, -0.0981,  ..., -0.0562, -0.0808,  0.0252],
       device='cuda:0')


In [46]:
output.shape

torch.Size([55, 200])

In [47]:
output

tensor([[-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087],
        [-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087],
        [-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087],
        ...,
        [-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087],
        [-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087],
        [-0.1526, -0.0742, -0.1011,  ...,  0.1077, -0.3130,  0.0087]],
       device='cuda:0')

In [53]:
relu

tensor([[0.0231, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0252],
        [0.0406, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0420],
        [0.1706, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0518],
        ...,
        [0.2850, 0.0000, 0.1183,  ..., 0.3028, 0.0285, 0.0000],
        [0.2803, 0.0000, 0.1239,  ..., 0.3090, 0.0399, 0.0000],
        [0.2471, 0.0000, 0.1255,  ..., 0.2441, 0.0518, 0.0000]],
       device='cuda:0')

In [8]:
def gen_preds(test, trading_data:torch_classes.TradingData, model:torch_classes.GRUNet):
    test['stats']  = pd.Series(test[stats_cols].fillna(-1).values.tolist())
    stock_ids = test.stock_id.unique().tolist()
    stocks = [trading_data.stocksDict[x] for x in stock_ids]
    hidden = torch.stack([trading_data.stocksDict[x].hidden for x in stock_ids]).transpose(0,1)

    print(hidden)
    
    X = [torch.tensor(x) for x in test['stats'].tolist()]
    X = torch.stack(X)
    X = pack_sequence(X.view(-1,1,12))
    
    hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

    output,hidden = model(X,hidden_in, p1=True)
    hidden = hidden.transpose(0,1)
    # output  = torch.flatten(output)

    [setattr(obj, 'hidden_test', val.detach()) for obj, val in zip(stocks,hidden)]

    [setattr(obj, 'hidden_out', val.detach()) for obj, val in zip(stocks,output)]

    stocks_hidden = trading_data.fetch_daily_data_test()

    print(stocks_hidden)

    X = torch.cat(stocks_hidden,dim=-1)
    output = model(X).tolist()[0]
    print(output)

    out = pd.Series(output)
    # print(out)

    out = out[out.index.isin(test['stock_id'])]
    # print(out)
    return out

In [9]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    if counter == 0:
        print(test.head(3))
        print(revealed_targets.head(3))
        print(sample_prediction.head(3))

        
    stock_ids = test.stock_id.unique().tolist()
    preditcions = gen_preds(test,trading_data,model)

    # preditcions = 
    
    test['predictions'] = preditcions
    # sample_prediction['actual'] = test['target']
    # print(preditcions)
    # test['']
    env.predict(test)
    counter += 1
    print(counter)
    # break

   Unnamed: 0  stock_id  date_id  seconds_in_bucket  imbalance_size  \
0           0         0        0                  0      3180602.69   
1           1         1        0                  0       166603.91   
2           2         2        0                  0       302879.87   

   imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                        1         0.999812   13380276.64        NaN   
1                       -1         0.999896    1642214.25        NaN   
2                       -1         0.999561    1819368.03        NaN   

   near_price  bid_price  bid_size  ask_price  ask_size  wap    target row_id  \
0         NaN   0.999812  60651.50   1.000026   8493.03  1.0 -3.029704  0_0_0   
1         NaN   0.999896   3233.04   1.000660  20605.09  1.0 -5.519986  0_0_1   
2         NaN   0.999403  37956.00   1.000298  18995.00  1.0 -8.389950  0_0_2   

   baseline_prediction  simple_prediction  
0                    0                0.1  
1            